### Import all necessary Libraries

In [2]:
!pip install wikipedia
#!pip install sentence_transformers

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=9adb259ffb81e21754a5a1c39ed77d6907767048b777ded20bffa14d2ebfe7c1
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.8 MB/s eta 0:00:00


In [4]:
import wikipedia
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

### Retrieve Knowledge

In [5]:
def get_wikipedia_content(topic):
  try:
    page = wikipedia.page(topic)
    return page.content
  except wikipedia.exceptions.PageError:
    return None
  except wikipedia.exceptions.DisambiguationError as e:
    # handle cases where the topic is ambiguous
    print(f"Ambiguous topic: Please be more specific. Options: {e.options}")
    return None

# user input
topic = input("Enter a topic: ")
wikipedia_content = get_wikipedia_content(topic)

if not wikipedia_content:
  print("No Wikipedia content found for the given topic.")
  exit()

Enter a topic: who found netflix


### split the text into smaller overlapping chunks for better retrieval

In [14]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

def split_text(text, chunk_size=256, chunk_overlap=20):
    tokens = tokenizer.tokenize(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunks.append(tokenizer.convert_tokens_to_string(tokens[start:end]))
        if end == len(tokens):
            break
        start = end - chunk_overlap
    return chunks

chunks = split_text(wikipedia_content)
print(f"Number of chunks: {len(chunks)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (14750 > 512). Running this sequence through the model will result in indexing errors


Number of chunks: 63


In [15]:
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = embedding_model.encode(chunks)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [ ]:
query = input("Ask a question about the topic: ")
query_embedding = embedding_model.encode([query])

k = 3
distances, indices = index.search(np.array(query_embedding), k)
retrieved_chunks = [chunks[i] for i in indices[0]]
print("Retrieved chunks:")
for chunk in retrieved_chunks:
    print("- " + chunk)

In [13]:
qa_model_name = "deepset/roberta-base-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)

context = " ".join(retrieved_chunks)
answer = qa_pipeline(question=query, context=context)
print(f"Answer: {answer['answer']}")

Device set to use cuda:0


Answer: marc randolph and reed hastings
